### Create a first-pass of glaciers that we believe should be included in the study.

We are going to identify prospective debris-covered glaciers in our study region (RGI v7.0 regions 13, 14, and 15). 

In order to be included, a glacier's RGI v7.0 outline must:
1. be larger in area than 2 km2
2. contain at least 1 km2 debris cover from herreid et al 2020 OR have at least 2 km2 of combined debris cover and 'missing' area from herreid 2020 (the glacier outlines used in herreid 2020 do not perfectly overlap the RGI7 outlines)

In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import pandas as pd
from datetime import timedelta
from datetime import datetime

In [105]:
# define folder and file paths
folder_HMA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"HMA2")
folder_glaciers = os.path.join(folder_HMA, 'Glacier Products')
fp_rgi = os.path.join(folder_HMA, 'Datasets', 'AOI definition', 'Final AOI', "Final Products", 'AOI_full_rgi.shp')
fp_dc = os.path.join(folder_HMA, 'Datasets', 'AOI definition', 'Final AOI', "Final Products", 'AOI_dc_rgi.shp')
folder_rgi6 = os.path.join(folder_HMA, "Datasets", "RGI", "v6")
folder_rgi = os.path.join(folder_HMA, 'Datasets', 'RGI')
fp_regions = os.path.join(folder_HMA, 'Datasets', 'AOI definition', 'Final AOI', "Final Products", 'subregion_boundaries.shp')

In [9]:
# open csvs that allow us to link rgiids between v6 and v7
link_13 = pd.read_csv(os.path.join(folder_rgi, 'links', 'RGI2000-v7.0-G-13_central_asia-rgi6_links.csv'))
link_14 = pd.read_csv(os.path.join(folder_rgi, 'links', 'RGI2000-v7.0-G-14_south_asia_west-rgi6_links.csv'))
link_15 = pd.read_csv(os.path.join(folder_rgi, 'links', 'RGI2000-v7.0-G-15_south_asia_east-rgi6_links.csv'))
link_all = pd.concat([link_13,link_14,link_15])

In [17]:
##### open our study glaciers rgi
rgi_gdf = gpd.read_file(fp_rgi).to_crs("ESRI:102025")
dc_gdf = gpd.read_file(fp_dc).to_crs("ESRI:102025")

In [23]:
# open rgiv6
rgi613 = gpd.read_file(os.path.join(folder_rgi6,"13.rgi60.CentralAsia", '13_rgi60_CentralAsia.shp')).to_crs("ESRI:102025")
rgi614 = gpd.read_file(os.path.join(folder_rgi6,"14.rgi60.SouthAsiaWest", '14_rgi60_SouthAsiaWest.shp')).to_crs("ESRI:102025")
rgi615 = gpd.read_file(os.path.join(folder_rgi6,"15.rgi60.SouthAsiaEast", '15_rgi60_SouthAsiaEast.shp')).to_crs("ESRI:102025")
rgi6all = gpd.GeoDataFrame(pd.concat([rgi613, rgi614, rgi615]))

In [51]:
# subset rgi6 to glacier that intersect with dc_gdf
join = gpd.sjoin(left_df=dc_gdf, right_df=rgi6all[['geometry', 'RGIId']], how="left")
rgi6_names = join['RGIId'].values
rgi6overlapglaciers = rgi6all[rgi6all['RGIId'].isin(rgi6_names)]

In [52]:
# save to visualize
out_path = os.path.join(folder_HMA, "rgi6glaciers.shp")
rgi6overlapglaciers.to_file(out_path)

In [112]:
# open subregions gdf, resave each as its own geom
regions_gdf = gpd.read_file(fp_regions).to_crs("EPSG:4326")
for i in range(22):
    subset = regions_gdf[regions_gdf['id']==i+1]
    subset.to_file(os.path.join(folder_HMA, 'Datasets', 'debris thickness',
                                'regions', f'r{str(i+1).zfill(2)}.json'), driver="GeoJSON")
# dataframe.to_file("output.json", driver="GeoJSON")

In [43]:
dc_gdf.shape

(769, 30)

In [45]:
# aggregate list of our glaciers
names_6 = join['RGIId'].values

In [46]:
names_6

array(['RGI60-13.01148', 'RGI60-13.01432', 'RGI60-13.01316',
       'RGI60-13.02655', 'RGI60-13.01956', 'RGI60-13.01863',
       'RGI60-13.00788', 'RGI60-13.01231', 'RGI60-13.01184',
       'RGI60-13.01392', 'RGI60-13.03762', 'RGI60-13.01937',
       'RGI60-13.00906', 'RGI60-13.01410', 'RGI60-13.01536',
       'RGI60-13.01178', 'RGI60-13.01027', 'RGI60-13.00940',
       'RGI60-13.01268', 'RGI60-15.01078', 'RGI60-13.00982',
       'RGI60-13.02647', 'RGI60-13.01969', 'RGI60-13.01980',
       'RGI60-13.01044', 'RGI60-13.00885', 'RGI60-13.01333',
       'RGI60-13.03308', nan, 'RGI60-13.01176', 'RGI60-13.02176',
       'RGI60-13.01228', 'RGI60-13.03307', 'RGI60-13.01456',
       'RGI60-13.01391', 'RGI60-13.01428', 'RGI60-13.03755',
       'RGI60-13.01326', 'RGI60-13.01500', 'RGI60-13.01417',
       'RGI60-13.01129', 'RGI60-13.02643', 'RGI60-14.18517',
       'RGI60-14.15894', 'RGI60-14.15895', 'RGI60-14.15966',
       'RGI60-14.15934', 'RGI60-14.15869', 'RGI60-14.15623',
       'RGI60-14.15

In [74]:
# use the link files to see what our glaciers used to be called
old_names = []

for idx,row in dc_gdf.iterrows():
    id7 = row['rgi_id']
    df_6 = link_all[link_all['rgi7_id']==id7]
    
    if len(df_6["overlap_area_km2"])<2:
        for i in df_6["rgi6_id"]:
            old_names.append(i)
    else:
        subset = df_6[df_6["overlap_area_km2"]>0.15]
        for i in subset["rgi6_id"]:
            old_names.append(i)
        

In [76]:
print(old_names[0])
len(old_names)

RGI60-15.00621


1579

In [79]:
format_names = [ f"HMA_DTE_{i[-8:]}" for i in old_names ]

In [80]:
format_names[0]

'HMA_DTE_15.00621'

In [84]:
test = ""
for i in format_names:
    test = test+","+i
test = test[1:]

In [87]:
len(test)

26842

In [94]:
len(test)/1700

15.789411764705882

In [102]:
test1 = test[:17*500]
test2 = test[17*500: 17*1000]
test3 = test[17*1000: 17*1500]
test4 = test[17*1500:]

In [96]:
test1

'HMA_DTE_15.00621,HMA_DTE_13.01148,HMA_DTE_13.01432,HMA_DTE_13.01316,HMA_DTE_13.02655,HMA_DTE_13.01955,HMA_DTE_13.01866,HMA_DTE_13.01956,HMA_DTE_13.01954,HMA_DTE_13.01863,HMA_DTE_13.00788,HMA_DTE_13.01228,HMA_DTE_13.00794,HMA_DTE_13.01231,HMA_DTE_13.01184,HMA_DTE_13.01394,HMA_DTE_13.01392,HMA_DTE_13.03872,HMA_DTE_13.03871,HMA_DTE_13.03762,HMA_DTE_13.01937,HMA_DTE_13.00906,HMA_DTE_13.01409,HMA_DTE_13.01410,HMA_DTE_13.01411,HMA_DTE_13.01524,HMA_DTE_13.01536,HMA_DTE_13.01392,HMA_DTE_13.00956,HMA_DTE_13.00955,HMA_DTE_13.01178,HMA_DTE_13.01027,HMA_DTE_13.00940,HMA_DTE_13.01268,HMA_DTE_15.00998,HMA_DTE_13.01130,HMA_DTE_15.01078,HMA_DTE_13.00982,HMA_DTE_13.00978,HMA_DTE_13.02647,HMA_DTE_13.01969,HMA_DTE_13.01980,HMA_DTE_13.01044,HMA_DTE_13.00885,HMA_DTE_13.01333,HMA_DTE_13.03308,HMA_DTE_13.01176,HMA_DTE_13.02176,HMA_DTE_13.01805,HMA_DTE_13.01926,HMA_DTE_13.01181,HMA_DTE_13.01228,HMA_DTE_13.01023,HMA_DTE_13.00761,HMA_DTE_13.03307,HMA_DTE_13.01456,HMA_DTE_13.01381,HMA_DTE_13.01384,HMA_DTE_13.01

In [98]:
test2

'HMA_DTE_15.09862,HMA_DTE_15.02867,HMA_DTE_15.03744,HMA_DTE_15.03201,HMA_DTE_15.10697,HMA_DTE_15.02240,HMA_DTE_15.02228,HMA_DTE_15.01151,HMA_DTE_15.03619,HMA_DTE_15.09803,HMA_DTE_15.09803,HMA_DTE_15.10866,HMA_DTE_15.10878,HMA_DTE_15.10860,HMA_DTE_15.01140,HMA_DTE_15.01191,HMA_DTE_15.01152,HMA_DTE_13.01430,HMA_DTE_15.09871,HMA_DTE_15.09803,HMA_DTE_15.02661,HMA_DTE_15.10968,HMA_DTE_13.26440,HMA_DTE_15.10883,HMA_DTE_15.01353,HMA_DTE_15.01354,HMA_DTE_15.01355,HMA_DTE_15.03765,HMA_DTE_15.03243,HMA_DTE_15.03206,HMA_DTE_15.09729,HMA_DTE_15.10135,HMA_DTE_15.10209,HMA_DTE_15.10232,HMA_DTE_15.02637,HMA_DTE_15.10814,HMA_DTE_15.04258,HMA_DTE_15.03731,HMA_DTE_15.03742,HMA_DTE_15.03889,HMA_DTE_15.03890,HMA_DTE_15.02833,HMA_DTE_15.02911,HMA_DTE_15.03518,HMA_DTE_15.03498,HMA_DTE_15.03524,HMA_DTE_15.03523,HMA_DTE_15.03522,HMA_DTE_15.09675,HMA_DTE_15.02235,HMA_DTE_15.02664,HMA_DTE_15.02326,HMA_DTE_15.02324,HMA_DTE_15.02325,HMA_DTE_15.02369,HMA_DTE_15.02696,HMA_DTE_15.10859,HMA_DTE_15.10866,HMA_DTE_15.02

In [103]:
test3

'HMA_DTE_13.05000,HMA_DTE_13.05019,HMA_DTE_13.05002,HMA_DTE_13.04962,HMA_DTE_13.04960,HMA_DTE_13.04965,HMA_DTE_13.04966,HMA_DTE_13.04967,HMA_DTE_13.04964,HMA_DTE_13.05038,HMA_DTE_13.04992,HMA_DTE_13.43613,HMA_DTE_13.30878,HMA_DTE_13.31120,HMA_DTE_13.31147,HMA_DTE_13.31151,HMA_DTE_13.31145,HMA_DTE_13.31154,HMA_DTE_13.38764,HMA_DTE_13.40583,HMA_DTE_13.40580,HMA_DTE_13.40536,HMA_DTE_13.37193,HMA_DTE_13.37183,HMA_DTE_13.37269,HMA_DTE_13.37271,HMA_DTE_13.37369,HMA_DTE_13.37370,HMA_DTE_13.40170,HMA_DTE_13.40796,HMA_DTE_13.38803,HMA_DTE_13.37368,HMA_DTE_13.38882,HMA_DTE_13.40704,HMA_DTE_13.40709,HMA_DTE_13.40696,HMA_DTE_13.40700,HMA_DTE_13.38924,HMA_DTE_13.37246,HMA_DTE_14.23523,HMA_DTE_14.24212,HMA_DTE_14.24206,HMA_DTE_14.24213,HMA_DTE_14.23140,HMA_DTE_14.20858,HMA_DTE_14.23441,HMA_DTE_14.23656,HMA_DTE_14.23649,HMA_DTE_14.23645,HMA_DTE_14.22225,HMA_DTE_14.22208,HMA_DTE_14.22310,HMA_DTE_14.22185,HMA_DTE_14.22276,HMA_DTE_14.22703,HMA_DTE_14.23757,HMA_DTE_14.24214,HMA_DTE_14.23787,HMA_DTE_14.23

In [104]:
test4

'HMA_DTE_14.17302,HMA_DTE_14.17300,HMA_DTE_14.18483,HMA_DTE_14.20031,HMA_DTE_14.20033,HMA_DTE_14.20030,HMA_DTE_14.14820,HMA_DTE_14.17250,HMA_DTE_14.20026,HMA_DTE_14.20025,HMA_DTE_14.20211,HMA_DTE_14.17248,HMA_DTE_14.18195,HMA_DTE_14.20156,HMA_DTE_14.20029,HMA_DTE_14.20210,HMA_DTE_14.19942,HMA_DTE_14.20162,HMA_DTE_14.20213,HMA_DTE_14.19394,HMA_DTE_14.20211,HMA_DTE_14.17289,HMA_DTE_14.18218,HMA_DTE_14.18977,HMA_DTE_14.19728,HMA_DTE_14.17266,HMA_DTE_14.17928,HMA_DTE_14.17265,HMA_DTE_14.17273,HMA_DTE_14.17910,HMA_DTE_14.17912,HMA_DTE_14.17911,HMA_DTE_14.20212,HMA_DTE_14.17642,HMA_DTE_14.17290,HMA_DTE_14.17285,HMA_DTE_14.17287,HMA_DTE_14.17630,HMA_DTE_14.18918,HMA_DTE_14.18975,HMA_DTE_14.20145,HMA_DTE_14.20144,HMA_DTE_14.21310,HMA_DTE_14.19759,HMA_DTE_14.17296,HMA_DTE_14.14970,HMA_DTE_14.14986,HMA_DTE_14.17304,HMA_DTE_14.17683,HMA_DTE_14.17682,HMA_DTE_14.18423,HMA_DTE_14.17281,HMA_DTE_14.17975,HMA_DTE_14.18466,HMA_DTE_14.18468,HMA_DTE_14.18467,HMA_DTE_14.17256,HMA_DTE_14.17064,HMA_DTE_14.16